# Data Question 2


## How is queue time impacted by RAM requested? How is it impacted by wall time requested?

## Are there any nodes that fail more than the other ones?


## Identify problematic users
Those people who submit greater than 500 jobs, with each less than 5 minutes within 4 hours

In [17]:
#import libraries 
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from datetime import datetime
%matplotlib inline

In [5]:
accre_df = pd.read_csv('Data/july-2017-thru-june-2018.txt', sep = '|', nrows = 10000)
accre_df.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,COMPLETED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21


In [6]:
accre_df.columns

Index(['JobID', 'JobIDRaw', 'Cluster', 'Partition', 'Account', 'Group', 'GID',
       'User', 'UID', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed',
       'ExitCode', 'State', 'NNodes', 'NCPUS', 'ReqCPUS', 'ReqMem', 'ReqGRES',
       'ReqTRES', 'Timelimit', 'NodeList', 'JobName'],
      dtype='object')

## Queue time = Start time - submit time

Queue time is the amount of time that a user has to wait until the job runs

## We want to see how requested RAM (reqMem) and requested wall time (Timelimit) affect the Queue time

In [14]:
# Get the data types and size of each of the columns
accre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
JobID        10000 non-null object
JobIDRaw     10000 non-null int64
Cluster      10000 non-null object
Partition    10000 non-null object
Account      10000 non-null object
Group        10000 non-null object
GID          10000 non-null int64
User         10000 non-null object
UID          8160 non-null float64
Submit       10000 non-null object
Eligible     10000 non-null object
Start        10000 non-null object
End          10000 non-null object
Elapsed      10000 non-null object
ExitCode     10000 non-null object
State        10000 non-null object
NNodes       10000 non-null int64
NCPUS        10000 non-null int64
ReqCPUS      10000 non-null int64
ReqMem       10000 non-null object
ReqGRES      38 non-null object
ReqTRES      10000 non-null object
Timelimit    10000 non-null object
NodeList     10000 non-null object
JobName      10000 non-null object
dtypes: float64(1), int64

## Submit and start times are both strings, so we need to convert those to a form that we can use

In [15]:
# Checking the components of the Eligible column
accre_df.Eligible.head()


0    2017-07-02T00:42:59
1    2017-07-02T00:42:59
2    2017-07-02T00:42:59
3    2017-07-02T00:42:59
4    2017-07-01T05:00:00
Name: Eligible, dtype: object

In [16]:
# Checking the components of the Submit column
accre_df.Submit.head()

0    2017-07-02T00:40:58
1    2017-07-02T00:40:58
2    2017-07-02T00:40:58
3    2017-07-02T00:40:58
4    2017-07-01T05:00:00
Name: Submit, dtype: object

In [20]:
# Change the submit and start column to a date time 64 data type
accre_df['Submit'] = pd.to_datetime(accre_df['Submit'], format="%Y/%m/%d")
accre_df['Start'] = pd.to_datetime(accre_df['Start'], format="%Y/%m/%d")

In [21]:
accre_df.Start.head()

0   2017-07-03 02:13:59
1   2017-07-03 02:20:08
2   2017-07-03 02:41:13
3   2017-07-03 02:51:53
4   2017-07-01 05:00:01
Name: Start, dtype: datetime64[ns]

## Subtract the submit time from the queue time to find the queue time

In [25]:
accre_df['queue_time'] = accre_df['Start'] - accre_df['Submit']
print(accre_df.queue_time.head(30))

0    1 days 01:33:01
1    1 days 01:39:10
2    1 days 02:00:15
3    1 days 02:10:55
4    0 days 00:00:01
5    0 days 00:00:03
6    0 days 00:00:02
7    0 days 00:00:01
8    0 days 00:00:01
9    0 days 00:00:03
10   0 days 00:01:43
11   0 days 00:23:01
12   0 days 00:36:42
13   0 days 01:01:42
14   0 days 01:25:36
15   0 days 01:40:07
16   0 days 02:01:42
17   0 days 03:01:45
18   0 days 03:41:41
19   0 days 03:51:59
20   0 days 04:00:01
21   0 days 05:30:14
22   0 days 05:36:41
23   0 days 05:46:40
24   0 days 05:58:32
25   0 days 07:26:40
26   0 days 07:35:36
27   0 days 07:47:14
28   0 days 08:26:39
29   0 days 08:46:39
Name: queue_time, dtype: timedelta64[ns]


In [24]:
accre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 26 columns):
JobID         10000 non-null object
JobIDRaw      10000 non-null int64
Cluster       10000 non-null object
Partition     10000 non-null object
Account       10000 non-null object
Group         10000 non-null object
GID           10000 non-null int64
User          10000 non-null object
UID           8160 non-null float64
Submit        10000 non-null datetime64[ns]
Eligible      10000 non-null object
Start         10000 non-null datetime64[ns]
End           10000 non-null object
Elapsed       10000 non-null object
ExitCode      10000 non-null object
State         10000 non-null object
NNodes        10000 non-null int64
NCPUS         10000 non-null int64
ReqCPUS       10000 non-null int64
ReqMem        10000 non-null object
ReqGRES       38 non-null object
ReqTRES       10000 non-null object
Timelimit     10000 non-null object
NodeList      10000 non-null object
JobName       10000 